In [2]:
# Data science helpers
from pathlib import Path
import pandas as pd 
import numpy as np

import featuretools as ft


2024-06-15 22:38:47,900 featuretools - WARNING    While loading primitives via "premium_primitives" entry point, ignored primitive "DiversityScore" from "premium_primitives.diversity_score" because a primitive with that name already exists in "nlp_primitives.diversity_score"
2024-06-15 22:38:47,901 featuretools - WARNING    While loading primitives via "premium_primitives" entry point, ignored primitive "LSA" from "premium_primitives.lsa" because a primitive with that name already exists in "nlp_primitives.lsa"
2024-06-15 22:38:47,901 featuretools - WARNING    While loading primitives via "premium_primitives" entry point, ignored primitive "MeanCharactersPerSentence" from "premium_primitives.mean_characters_per_sentence" because a primitive with that name already exists in "nlp_primitives.mean_characters_per_sentence"
2024-06-15 22:38:47,901 featuretools - WARNING    While loading primitives via "premium_primitives" entry point, ignored primitive "NumberOfSentences" from "premium_primi

In [3]:
# Read in all data
df_test = pd.read_csv("test.csv")
df_sub = pd.read_csv("sample_submission.csv")
df_stores = pd.read_csv("stores.csv")
df_items = pd.read_csv("items.csv")
df_transactions = pd.read_csv("transactions.csv", parse_dates=['date'])
df_oil = pd.read_csv("oil.csv", parse_dates=['date'])
df_holiday = pd.read_csv("holidays_events.csv", parse_dates=['date'])

In [4]:

df_train = pd.read_csv("train.csv",  parse_dates=['date']  )

/var/folders/gq/789ffg0j3_s7scj27n_mdkj40000gn/T/ipykernel_11344/932450269.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv("train.csv",  parse_dates=['date']  )


In [5]:
print(df_train.head())

   id       date  store_nbr  item_nbr  unit_sales onpromotion
0   0 2013-01-01         25    103665         7.0         NaN
1   1 2013-01-01         25    105574         1.0         NaN
2   2 2013-01-01         25    105575         2.0         NaN
3   3 2013-01-01         25    108079         1.0         NaN
4   4 2013-01-01         25    108701         1.0         NaN


In [6]:
train_items=df_train.merge(df_items, left_on='item_nbr', right_on='item_nbr',
                 how='left' )
train_items=train_items.merge(df_holiday,left_on='date', right_on='date',
                 how='left' )
train_items=train_items.merge(df_oil,left_on='date', right_on='date',
                 how='left' )

In [7]:
train_items.head()
train_items=train_items.dropna()

In [8]:
df_transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [9]:
all_info=train_items.merge(df_transactions, left_on=['date','store_nbr'], right_on=['date','store_nbr'],                how='left' )

In [10]:
all_info=all_info.merge(df_stores, left_on=['store_nbr'] ,right_on=['store_nbr'],
           how='left' )

In [11]:
all_info['date']=pd.to_datetime(all_info['date'],format= "%d/%m/%Y")

In [12]:
all_info['month']=all_info['date'].dt.month

In [13]:
all_info['year']=all_info['date'].dt.year

In [14]:
all_info['day']=all_info['date'].dt.weekday

In [15]:
dummy_variables = ['onpromotion','type_x','cluster',
                'family','perishable','type_y','onpromotion', 'transferred']

In [16]:
dummies=pd.get_dummies(all_info[dummy_variables])

In [17]:
all_info_no_dummies = all_info.drop(dummy_variables, axis=1)

In [18]:
df=all_info_no_dummies.merge(dummies,left_index=True,right_index=True)

In [19]:
df.columns

Index(['id', 'date', 'store_nbr', 'item_nbr', 'unit_sales', 'class', 'locale',
       'locale_name', 'description', 'dcoilwtico', 'transactions', 'city',
       'state', 'month', 'year', 'day', 'cluster', 'perishable',
       'onpromotion_False', 'onpromotion_True', 'type_x_Additional',
       'type_x_Bridge', 'type_x_Event', 'type_x_Holiday', 'type_x_Transfer',
       'family_AUTOMOTIVE', 'family_BABY CARE', 'family_BEAUTY',
       'family_BEVERAGES', 'family_BOOKS', 'family_BREAD/BAKERY',
       'family_CELEBRATION', 'family_CLEANING', 'family_DAIRY', 'family_DELI',
       'family_EGGS', 'family_FROZEN FOODS', 'family_GROCERY I',
       'family_GROCERY II', 'family_HARDWARE', 'family_HOME AND KITCHEN I',
       'family_HOME AND KITCHEN II', 'family_HOME APPLIANCES',
       'family_HOME CARE', 'family_LADIESWEAR', 'family_LAWN AND GARDEN',
       'family_LINGERIE', 'family_LIQUOR,WINE,BEER', 'family_MAGAZINES',
       'family_MEATS', 'family_PERSONAL CARE', 'family_PET SUPPLIES',
    

In [20]:
%store df

Stored 'df' (DataFrame)


In [21]:
df = df.loc[:,~df.columns.duplicated()].copy()


In [22]:
df

,id,date,store_nbr,item_nbr,unit_sales,class,locale,locale_name,description,dcoilwtico,...,family_PRODUCE,family_SCHOOL AND OFFICE SUPPLIES,family_SEAFOOD,type_y_A,type_y_B,type_y_C,type_y_D,type_y_E,transferred_False,transferred_True
0,21657651,2014-04-01,1,103520,1.0,1028,Regional,Cotopaxi,Provincializacion de Cotopaxi,99.69,...,False,False,False,False,False,False,True,False,True,False
1,21657652,2014-04-01,1,103665,3.0,2712,Regional,Cotopaxi,Provincializacion de Cotopaxi,99.69,...,False,False,False,False,False,False,True,False,True,False
2,21657653,2014-04-01,1,105574,1.0,1045,Regional,Cotopaxi,Provincializacion de Cotopaxi,99.69,...,False,False,False,False,False,False,True,False,True,False
3,21657654,2014-04-01,1,105575,11.0,1045,Regional,Cotopaxi,Provincializacion de Cotopaxi,99.69,...,False,False,False,False,False,False,True,False,True,False
4,21657655,2014-04-01,1,105577,4.0,1045,Regional,Cotopaxi,Provincializacion de Cotopaxi,99.69,...,False,False,False,False,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12560595,125497035,2017-08-15,54,2089339,4.0,1006,Local,Riobamba,Fundacion de Riobamba,47.57,...,False,False,False,False,False,True,False,False,True,False
12560596,125497036,2017-08-15,54,2106464,1.0,1148,Local,Riobamba,Fundacion de Riobamba,47.57,...,False,False,False,False,False,True,False,False,True,False
12560597,125497037,2017-08-15,54,2110456,192.0,1120,Local,Riobamba,Fundacion de Riobamba,47.57,...,False,False,False,False,False,True,False,False,True,False
12560598,125497038,2017-08-15,54,2113914,198.0,3040,Local,Riobamba,Fundacion de Riobamba,47.57,...,False,False,False,False,False,True,False,False,True,False
